## Método Empírico Para Diseño de Mallas de Producción

**Formulismo Método Pearse-Borquéz**


$$Burden:\, B = 0.0254 \, \cdot \, Kv \, \cdot \, D \, \cdot \, \frac{Pd}{Ts}^{0.5} (pulgadas) $$

$$Indice\, de\, volabilidad:\,Kv = 1.96 \, - 0.27\, \cdot \, ln(RQED)$$

$$Indice\, de\, calidad\, de\, roca \,equivalente:\,RQED = Fc\, \cdot \,RQD$$

**Formulismo Taco Crítico Método de Chiappetta:**

$$Profundidad\, de\, entierro\, escalada:\, SD = \frac{D}{W^{0.33}} (metros)$$

$$Distancia\, superficie\, al\, centro\, de\, carga:\, D = Tcritico + \frac{1}{2}\, \cdot \,lsd\, (metros)$$

$$Masa\, explosiva:\, W = Ql \cdot \,lsd\, (Kg)$$

$$Explosivo\, por\, metro:\, Ql = dExplosivo(gr/cc)\, \cdot \,Dper(pulgadas) \cdot\ 0.507 (Kg/m)$$

$$Taco\, critico:\, Tcritico = SD\, \cdot \,W^{0.33} \,- \frac{1}{2}\, \cdot \,lsd\, (metros)$$

In [1]:
#Antecedentes:

d = 12                                            # Diametro del pozo (pulg)
de = 1.1                                         # Densidad explosivo (t/m3)
vod = 3947                                       # VOD (m/s)
h= 12                                            # Altura banco (metros)
dr = 2.7                                         # Densidad roca (t/m3)
rqd = 60                                         # RQD ( %)
ucs = 1529.58                                    # UCS (PSI) resistencia a la compresion
srb = 1                                          # SBR  (B/E): 1
rd = 0.7                                         # Resistencia de las discontinuidades = muy baja (factor corrección 0.7)

#Dimensiones de la malla:

x_inicio = 0; x_final = 80; y_inicio = 0; y_final = 30

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

#Cálculos Método Pearse-Borquéz:

ts = (8/100)*ucs                                           # Ts (psi) resistencia a la traccion estatica
kv = (1.96 - 0.27*np.log(0.7*rqd)).round(1)                # Kv indice de volabilidad 
pod = (250*de*(vod**2))*0.000001                           #Pd (MPa) Potencia  de detonacion
pod_psi = 145.038* pod                                     # Pd (psi) Potencia  de detonacion
b = int(((0.0254*kv*d*((pod_psi/ts)**0.5))*0.3048).round(0))    #Burden en metros
e = b                                                      #Espaciamiento en metros

#Cálculos Taco Crítico de Chiappetta:

d_pozo = d*0.0254                                     # Diametro del pozo (m)
lsd = d_pozo*10                                       # Largo de la carga, equivalente 10*d_pozo (m)
ql = 0.507*de*d**2                                    # Cantidad de explosivo contenida en un metro (Kg/m)
w = lsd*ql                                            #Masa explosiva (Kg)
SD = 1.1                                              #Profundidad de entierro escalada (0.92 - 1.4)
Tcritico = int(round(((SD*(w**0.33))-(0.5*lsd)),0))   #Longitud del taco
j = 0.2*b                                             #Pasadura 

burden = int(b*srb)
espaciamiento= e
x_p = int(x_final/e+1)
y_p = int(y_final/b+1)
n_pozos = x_p*y_p
m_per = n_pozos*(h+j)
c_e = (h+j)-Tcritico
espaciamiento_x = []
for x in range(x_inicio, x_final, espaciamiento):
     espaciamiento_x.append(x)  
burden_y = []
for x in range(y_inicio, y_final, burden):
     burden_y.append(x)    
n_burden = len(burden_y)
n_espaciamiento = len(espaciamiento_x)
coordinates = []
for x in range(x_inicio, x_final, espaciamiento):
    for y in range(y_inicio, y_final, burden):
        coordinates.append((x, y))    
df = pd.DataFrame(coordinates)
df.rename(columns={0:'x',1:'y'}, inplace=True)
z = []
for x in np.arange(0.0, (h+j), 0.1):
     z.append(x)
a = z*810
df_z = pd.DataFrame(a)
df_z.rename(columns={0:'z'}, inplace=True)
newdf = pd.DataFrame(np.repeat(df.values,(h+j)*10,axis=0))
newdf.columns = df.columns
df2 = newdf.copy()
df2["z"] = df_z["z"]
# 0 = taco
# 1 = carga columna
#2 = carga de fondo
def f(row):
    if row['z'] >= h-Tcritico:
        val = "Taco"
    elif row['z']<= j:
        val = "Pasadura"
    else:
        val = "Explosivo"
    return val
df2['Carga de pozo'] = df2.apply(f, axis=1)

fig = px.scatter_3d(df2, x="x", y="y",z="z", color="Carga de pozo", 
                    color_discrete_sequence=px.colors.qualitative.Plotly)

fig.update_traces(marker=dict(size=5, symbol = 'circle',
                              line=dict(width=0,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="rgb(230, 230,200)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",),),
                    width=700,
                    margin=dict(
                    r=10, l=10,
                    b=10, t=10)
                  )
fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=8, range=[-5,x_final],),
                     yaxis = dict(nticks=8, range=[-5,y_final],),
                     zaxis = dict(nticks=10, range=[0,h+j],),),
    width=1000,
    margin=dict(r=20, l=10, b=10, t=10))
fig.update_layout(
    scene = dict(
        aspectratio = dict(
            x = x_final/50,
            y = y_final/50,
            z = (h+j)/50
        )
    ),
    margin = dict(
        t = 20,
        b = 20,
        l = 20,
        r = 20
    )
)
fig.update_layout(scene = dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z'),
                    width=900,
                    margin=dict(r=20, b=10, l=10, t=10))
fig.show()

data = {'Variable':["N° pozos", "Diámetro pozo", "Altura banco", 'Largo malla', 'Ancho malla', "Metros perforados", "Densidad Roca", 
                    "Densidad Explosivo","VOD", "RQD", "Resistencia a la compresión UCS",'Resistencia a la tracción estática', 
                    'Índice de volabilidad ', "Potencia  de detonación", "Burden", "Espaciamiento", "Taco", "Pasadura", "Carga explosiva"], 
        'Valor':[n_pozos, d_pozo, h, x_final, y_final, m_per, dr, de, vod, rqd,ucs,ts, kv, pod_psi, burden, e, Tcritico, j,c_e], 
        "Unidad": ["", "m","m","m", "m", "m", "t/m3","t/m3","m/s","%","psi","psi", "", "psi", "m", "m","m","m","m"]} 
  
df = pd.DataFrame(data) 
  
df 

,Variable,Valor,Unidad
0,N° pozos,60.000000,
1,Diámetro pozo,0.304800,m
2,Altura banco,12.000000,m
3,Largo malla,80.000000,m
4,Ancho malla,30.000000,m
5,Metros perforados,804.000000,m
6,Densidad Roca,2.700000,t/m3
7,Densidad Explosivo,1.100000,t/m3
8,VOD,3947.000000,m/s
9,RQD,60.000000,%
